In [1]:
from osim.env import L2RunEnv
from osim.env import ProstheticsEnv


In [2]:
env = L2RunEnv(visualize=True)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [11]:
observation = env.reset(project=True)
env.re

TypeError: reset() got an unexpected keyword argument 'difficulty'

In [10]:
observation

[-0.05,
 0.0,
 0.91,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.007169537779780744,
 1.5365721883823606,
 0.0,
 0.91,
 -0.09650084892621281,
 0.9964310485677471,
 0.007987580127344573,
 -0.027441466796053905,
 0.007987580127344573,
 -0.027441466796053905,
 -0.11968333174236659,
 0.022952398528571172,
 -0.11968333174236659,
 0.022952398528571172,
 -0.06973431783842708,
 0.9707692842528318,
 0.0,
 0.0,
 0.0,
 0.0,
 0,
 0,
 0,
 0,
 0]

In [8]:
observation

{'joint_pos': {'ground_pelvis': [-0.05, 0.0, 0.91],
  'hip_r': [0.0],
  'knee_r': [0.0],
  'ankle_r': [0.0],
  'subtalar_r': [],
  'mtp_r': [],
  'hip_l': [0.0],
  'knee_l': [0.0],
  'ankle_l': [0.0],
  'subtalar_l': [],
  'mtp_l': [],
  'back': [-0.139626],
  'back1': []},
 'joint_vel': {'ground_pelvis': [0.0, 0.0, 0.0],
  'hip_r': [0.0],
  'knee_r': [0.0],
  'ankle_r': [0.0],
  'subtalar_r': [],
  'mtp_r': [],
  'hip_l': [0.0],
  'knee_l': [0.0],
  'ankle_l': [0.0],
  'subtalar_l': [],
  'mtp_l': [],
  'back': [0.0],
  'back1': []},
 'joint_acc': {'ground_pelvis': [24.195893055832826,
   11.631283304067308,
   3.258895326698422],
  'hip_r': [-166.3421230159185],
  'knee_r': [259.57822281558106],
  'ankle_r': [2925.2721850812236],
  'subtalar_r': [],
  'mtp_r': [],
  'hip_l': [-166.3421230159185],
  'knee_l': [259.57822281558106],
  'ankle_l': [2925.2721850812236],
  'subtalar_l': [],
  'mtp_l': [],
  'back': [1.5987211554602254e-14],
  'back1': []},
 'body_pos': {'pelvis': [0.0, 0.91

In [22]:
dir(env.osim_model.get_joint('ankle_l'))

['PrintPropertyInfo',
 'PropertyIndex_socket_child_frame_connectee_name',
 'PropertyIndex_socket_parent_frame_connectee_name',
 'RegisterType',
 'RenameType',
 'SafeCopy',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__swig_getmethods__',
 '__swig_setmethods__',
 '__weakref__',
 '_has_output_power',
 '_has_output_reaction_on_child',
 '_has_output_reaction_on_parent',
 '_markAdopted',
 '_s',
 'addComponent',
 'addToSystem',
 'append_coordinates',
 'append_frames',
 'assign',
 'calcEquivalentSpatialForce',
 'calcPower',
 'calcReactionOnChildExpressedInGround',
 'calcReactionOnParentExpressedInGround',
 'clearConnections',
 'clone',
 '

In [94]:
env.osim_model.state_desc['joint_pos']['ankle_l']

[-0.5971462498078045]

In [14]:
env.osim_model.bodies

[]

In [4]:
env.model_path

'C:\\Users\\clvco\\Anaconda3\\envs\\opensim-rl\\lib\\site-packages\\osim\\env\\../models/gait9dof18musc.osim'

In [1]:
import opensim

In [5]:
model = opensim.Model(env.model_path)

In [7]:
model.initSystem()

<opensim.simbody.State; proxy of <Swig Object of type 'SimTK::State *' at 0x000001768E63EF60> >

In [16]:
model.getForceSet().get(0).getName()

'hamstrings_r'

In [98]:
reward

0.004674955096276773

In [100]:
reward

0.006222903911480282

In [99]:

# env = L2RunEnv(visualize=True)
# observation = env.reset()

for i in range(50):
    ac = env.action_space.sample()
    ac[:] = 0
    ac[17] = 1
    
    observation, reward, done, info = env.step(ac)
    print(done)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True


SystemError: <built-in function Manager_integrate> returned a result with an error set

In [3]:
def get_zeta(overshoot):
    x = math.log(overshoot)
    x = x/math.sqrt(math.log(overshoot)**2 + math.pi**2)
    print("ZETA: ", -x)
    return -x

def get_omega(tp, zeta):
    wn = math.pi/(tp*math.sqrt(1-zeta**2))
    print("OMEGA_N", wn)
    return wn

def get_tau(zeta, omega_n):
    t = 1/(2*zeta*omega_n)
    print("TAU", t)
    return t

def get_k1(kp, omega_n, tau):
    k1 = (omega_n**2)*tau/kp
    print("K1", k1)
    return k1

In [8]:
0.02*0.6

0.012

In [16]:
mag_target = 0.4
mag_actual = 0.2 + 0.232
square_start = 5
square_end = 5.118
KP = 11.75
OS = (mag_actual - mag_target)/mag_target
print("OS:", OS)
zeta = get_zeta(OS)
peaktime = square_end - square_start
print("PEAKTIME:", peaktime)
wn = get_omega(peaktime, zeta)
tau = get_tau(zeta, wn)
get_k1(KP, wn, tau)

OS: 0.08000000000000007
ZETA:  0.6265771868559203
PEAKTIME: 0.11800000000000033
OMEGA_N 34.16096290373316
TAU 0.02335959570833432
K1 2.319998280176458


2.319998280176458

In [24]:
2.188216135490001

TAU 0.1176887277761934


K1 0.4693984009490294


0.4693984009490294